In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
#alert_time train
alert_time = pd.read_csv('./training_first/train_x_alert_date.csv')
#alert_time test
test_alert_time = pd.read_csv('./training_first/public_x_alert_date.csv')
#y
y = pd.read_csv('./training_first/train_y_answer.csv')

#ccba
ccba = pd.read_csv('./training_first/public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv('./training_first/public_train_x_cdtx0001_full_hashed.csv')
#custinfo
custinfo = pd.read_csv('./training_first/public_train_x_custinfo_full_hashed.csv')
#dp
dp = pd.read_csv('./training_first/public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv('./training_first/public_train_x_remit1_full_hashed.csv')

#案件名單
doc = pd.read_csv('./training_first/預測的案件名單及提交檔案範例.csv')

In [3]:
#合併alert_key、date與label
#occupation_code missing補類別21
alert = pd.concat([alert_time,test_alert_time])
base = alert.merge(y, on = 'alert_key', how = 'left').merge(custinfo, on = 'alert_key', how = 'left')
base[['occupation_code']] = base[['occupation_code']].fillna(value=21.0)

In [4]:
#gen yymm
yymm = [0, 30, 61, 91, 122, 153, 183, 214, 244, 275, 306, 334, 365]
bins = [-np.inf, 29, 60, 90, 121, 152, 182, 213, 243, 274, 305, 333, 364, 393]
base['byymm'] = pd.cut(base['date'], bins, labels = yymm)
base

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,byymm
0,171189,0,0.0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4,0
1,171202,0,0.0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2,0
2,171599,0,0.0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4,0
3,171737,0,0.0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4,0
4,171142,0,0.0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3,0
...,...,...,...,...,...,...,...,...,...
25746,364472,393,NaN,bdda3ef79c5f052fd281c9f28f50a25977444eceaf9751...,3,19.0,2390889.0,4,365
25747,364788,393,NaN,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,365
25748,364673,393,NaN,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,365
25749,364626,393,NaN,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,365


In [5]:
#training data只取每個月最後一筆，testing data取全部
base1 = base[~base.sar_flag.isna()].sort_values(['cust_id','date']).drop_duplicates(subset=['cust_id','byymm'], keep='last')
base2 = base[base.sar_flag.isna()].sort_values(['cust_id','date']).drop_duplicates(subset=['cust_id','byymm'], keep='last')
base3 = pd.concat([base1,base2])
#排掉第1個月資料(dp資料不完整)
base3 = base3[base3.byymm>0]

<font size=5> CCBA

In [6]:
temp_p = base3[['cust_id','date','sar_flag','occupation_code','AGE']].sort_values(['cust_id','date']).drop_duplicates(subset=['cust_id'], keep='last')
temp_p_oc = temp_p[['occupation_code','sar_flag']].groupby('occupation_code', as_index=False).agg(occupation_code_tge=('sar_flag', 'mean'))
temp_p_age = temp_p[['AGE','sar_flag']].groupby('AGE', as_index=False).agg(AGE_tge=('sar_flag', 'mean'))
#occupation_code target encoding
base3 = base3.merge(temp_p_oc,on = 'occupation_code',how = 'left').merge(temp_p_age,on = 'AGE',how = 'left')

In [7]:
base3 = base3.merge(ccba, on=['cust_id','byymm'], how='left')

In [8]:
#當月是否有溢繳
base3['feature_01'] = base3.usgam.apply(lambda x: 1 if x < 0 else 0)
#當月額度使用率
base3['feature_02'] = base3.usgam / base3.cycam
#當月消費分期佔消費金額佔比
base3['feature_03'] = base3.inamt / base3.cucsm
#本月消費金額較前月金額成長率
def f(x):
    if x[0] == 0:
        return 999
    else:
        return (x[1] - x[0]) / x[0]
base3['feature_04'] = base3[['lupay','cucsm']].apply(f,axis=1)   

In [9]:
#是否無信用卡聯徵資料
base3['feature_05'] = base3.feature_01.apply(lambda x: 0 if x >= 0 else 1)
#risk_rank one hot
risk_rank_dummy = pd.get_dummies(base3['risk_rank'])
risk_rank_dummy.rename(columns={0:'risk_rank_0',1:'risk_rank_1',2:'risk_rank_2',3:'risk_rank_3'}, inplace=True)
base3 = pd.concat([base3,risk_rank_dummy], axis = 1)
base3

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,byymm,occupation_code_tge,...,cucah,feature_01,feature_02,feature_03,feature_04,feature_05,risk_rank_0,risk_rank_1,risk_rank_2,risk_rank_3
0,195913,53,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,30,0.037871,...,0.0,0,0.165510,0.363804,-0.477980,0,0,1,0,0
1,271556,221,0.0,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,3,4.0,629787.0,3,214,0.022989,...,0.0,0,0.701614,0.053334,999.000000,0,0,0,0,1
2,240088,153,0.0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,3,15.0,2126903.0,4,153,0.069364,...,0.0,0,0.081855,0.251057,-0.497005,0,0,0,0,1
3,294760,263,0.0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,3,15.0,980597.0,4,244,0.069364,...,0.0,0,0.078201,0.000000,-0.498427,0,0,0,0,1
4,206471,78,0.0,0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...,2,13.0,406.0,5,61,0.058824,...,0.0,0,0.978046,0.079825,-0.455419,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8803,364767,393,NaN,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,2,17.0,41066.0,3,365,0.037871,...,0.0,0,0.091587,0.438364,-0.415534,0,0,0,1,0
8804,355182,375,NaN,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,1,18.0,824.0,3,365,0.048309,...,0.0,0,0.732074,0.323301,-0.353585,0,0,1,0,0
8805,360492,386,NaN,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,1,19.0,2406614.0,6,365,0.038766,...,0.0,0,0.010380,0.000000,0.087237,0,0,1,0,0
8806,356426,377,NaN,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,3,19.0,21647.0,4,365,0.038766,...,NaN,0,NaN,NaN,NaN,0,0,0,0,1


<font size=5> DP

In [10]:
#串dp近30天交易資料
base3['start_date'] = base3.date - 30
base3['L1W_date'] = base3.date - 7
base3['L2W_date'] = base3.date - 14
base3['L3W_date'] = base3.date - 21
base3['L4W_date'] = base3.date - 28
dp_base = base3[['alert_key','date','cust_id','start_date','L1W_date','L2W_date','L3W_date','L4W_date']].merge(dp, on = 'cust_id', how = 'inner')
dp_base = dp_base[(dp_base.tx_date > dp_base.start_date) & (dp_base.tx_date <= dp_base.date)]

In [33]:
dp_base[dp_base.tx_date>dp_base.L1W_date]

,alert_key,date,cust_id,start_date,L1W_date,L2W_date,L3W_date,L4W_date,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM
17,195913,53,0015cc01f553512007705edf855baa16f1b57a16fb107e...,23,46,39,32,25,DB,53,18,2,142.0,21.55,20,NaN,NaN,0,1
19,195913,53,0015cc01f553512007705edf855baa16f1b57a16fb107e...,23,46,39,32,25,CR,53,16,2,1161.0,1.00,6,NaN,NaN,0,0
22,195913,53,0015cc01f553512007705edf855baa16f1b57a16fb107e...,23,46,39,32,25,DB,53,18,2,10407.0,21.55,20,NaN,NaN,0,1
87,195913,53,0015cc01f553512007705edf855baa16f1b57a16fb107e...,23,46,39,32,25,DB,47,11,2,826.0,1.00,13,22.0,99.0,0,1
90,195913,53,0015cc01f553512007705edf855baa16f1b57a16fb107e...,23,46,39,32,25,CR,53,18,2,2064.0,21.55,20,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4809470,360492,386,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,356,379,372,365,358,DB,386,12,2,5096.0,1.00,12,NaN,NaN,0,0
4809472,360492,386,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,356,379,372,365,358,DB,382,1,2,18346.0,1.00,16,NaN,NaN,0,0
4809474,360492,386,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,356,379,372,365,358,CR,386,12,2,1529.0,1.00,12,NaN,NaN,0,0
4809476,360492,386,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,356,379,372,365,358,DB,386,14,2,306.0,1.00,12,NaN,NaN,0,0


In [11]:
#近一周交易差額比率
session_amt_diff = dp_base[dp_base.tx_date>dp_base.L1W_date].groupby(['cust_id','date','debit_credit'])['tx_amt'].sum().reset_index()
session_amt_diff = pd.pivot_table(session_amt_diff, index=['cust_id','date'], columns='debit_credit', values='tx_amt')
session_amt_diff.fillna(0, inplace=True)
session_amt_diff['session_amt_diff_ratio'] = \
abs(session_amt_diff['CR'] - session_amt_diff['DB']) / abs(session_amt_diff['CR'] + session_amt_diff['DB'])
session_amt_diff = session_amt_diff.reset_index()[['cust_id','date','session_amt_diff_ratio']]
base3 = base3.merge(session_amt_diff, on=['cust_id','date'], how='left')
base3[['session_amt_diff_ratio']] = base3[['session_amt_diff_ratio']].fillna(-999)

In [12]:
#當日 交易差額比率
date_amt_diff = dp_base[dp_base.tx_date==dp_base.date].groupby(['cust_id','date','debit_credit'])['tx_amt'].sum().reset_index()
date_amt_diff
date_amt_diff = pd.pivot_table(date_amt_diff, index=['cust_id','date'], columns='debit_credit', values='tx_amt')
date_amt_diff.fillna(0, inplace=True)
date_amt_diff['date_amt_diff_ratio'] = \
abs(date_amt_diff['CR'] - date_amt_diff['DB']) / abs(date_amt_diff['CR'] + date_amt_diff['DB'])
date_amt_diff = date_amt_diff.reset_index()[['cust_id','date','date_amt_diff_ratio']]
base3 = base3.merge(date_amt_diff, on=['cust_id','date'], how='left')
base3[['date_amt_diff_ratio']] = base3[['date_amt_diff_ratio']].fillna(-999)

In [13]:
#當日交易筆數 tx_cnt_date
tx_cnt_date = dp_base[dp_base.tx_date==dp_base.date].groupby(['cust_id','date'])['debit_credit'].count().reset_index()
tx_cnt_date.rename(columns={'debit_credit':'tx_cnt_date'}, inplace=True)
base3 = base3.merge(tx_cnt_date, on=['cust_id','date'], how='left')
base3[['tx_cnt_date']] = base3[['tx_cnt_date']].fillna(0)
#當session交易筆數 tx_cnt_session
tx_cnt_session = dp_base[dp_base.tx_date>dp_base.L1W_date].groupby(['cust_id','date'])['debit_credit'].count().reset_index()
tx_cnt_session.rename(columns={'debit_credit':'tx_cnt_session'}, inplace=True)
base3 = base3.merge(tx_cnt_session, on=['cust_id','date'], how='left')
base3[['tx_cnt_session']] = base3[['tx_cnt_session']].fillna(0)
#單日總分行數 txbranch_day_cnt
txbranch_day_cnt = dp_base[dp_base.tx_date==dp_base.date].groupby(['cust_id','date'])['txbranch'].count().reset_index()
txbranch_day_cnt.rename(columns={'txbranch':'txbranch_day_cnt'}, inplace=True)
base3 = base3.merge(txbranch_day_cnt, on=['cust_id','date'], how='left')
base3[['txbranch_day_cnt']] = base3[['txbranch_day_cnt']].fillna(0)
#當session總分行數 txbranch_session_cnt
txbranch_session_cnt = dp_base[dp_base.tx_date>dp_base.L1W_date].groupby(['cust_id','date'])['txbranch'].count().reset_index()
txbranch_session_cnt.rename(columns={'txbranch':'txbranch_session_cnt'}, inplace=True)
base3 = base3.merge(txbranch_session_cnt, on=['cust_id','date'], how='left')
base3[['txbranch_session_cnt']] = base3[['txbranch_session_cnt']].fillna(0)

In [14]:
#當sessionATM 佔交易數比例
session_atm_txn_ratio = dp_base[dp_base.tx_date>dp_base.L1W_date].groupby(['cust_id','date'])['ATM'].sum().reset_index()
session_atm_txn_ratio.rename(columns={'ATM':'session_atm_txn_ratio'}, inplace=True)
tx_cnt_session_temp = tx_cnt_session.merge(session_atm_txn_ratio, on=['cust_id','date'], how='left')
tx_cnt_session_temp.session_atm_txn_ratio = tx_cnt_session_temp.session_atm_txn_ratio / tx_cnt_session_temp.tx_cnt_session
base3 = base3.merge(tx_cnt_session_temp.drop('tx_cnt_session',axis = 1), on=['cust_id','date'], how='left')
base3[['session_atm_txn_ratio']] = base3[['session_atm_txn_ratio']].fillna(0)
#當日ATM 佔交易數比例
day_atm_txn_ratio = dp_base[dp_base.tx_date==dp_base.date].groupby(['cust_id','date'])['ATM'].sum().reset_index()
day_atm_txn_ratio.rename(columns={'ATM':'day_atm_txn_ratio'}, inplace=True)
tx_cnt_date_temp = tx_cnt_date.merge(day_atm_txn_ratio, on=['cust_id','date'], how='left')
tx_cnt_date_temp.day_atm_txn_ratio = tx_cnt_date_temp.day_atm_txn_ratio / tx_cnt_date_temp.tx_cnt_date
base3 = base3.merge(tx_cnt_date_temp.drop('tx_cnt_date',axis = 1), on=['cust_id','date'], how='left')
base3[['day_atm_txn_ratio']] = base3[['day_atm_txn_ratio']].fillna(0)

In [15]:
#當session跨行 佔交易數比例
session_cross_bank_ratio = dp_base[dp_base.tx_date>dp_base.L1W_date].groupby(['cust_id','date'])['cross_bank'].sum().reset_index()
session_cross_bank_ratio.rename(columns={'cross_bank':'session_cross_bank_ratio'}, inplace=True)
tx_cnt_session_temp = tx_cnt_session.merge(session_cross_bank_ratio, on=['cust_id','date'], how='left')
tx_cnt_session_temp.session_cross_bank_ratio = tx_cnt_session_temp.session_cross_bank_ratio / tx_cnt_session_temp.tx_cnt_session
base3 = base3.merge(tx_cnt_session_temp.drop('tx_cnt_session',axis = 1), on=['cust_id','date'], how='left')
base3[['session_cross_bank_ratio']] = base3[['session_cross_bank_ratio']].fillna(0)
#當日跨行 佔交易數比例
day_cross_bank_ratio = dp_base[dp_base.tx_date==dp_base.date].groupby(['cust_id','date'])['cross_bank'].sum().reset_index()
day_cross_bank_ratio.rename(columns={'cross_bank':'day_cross_bank_ratio'}, inplace=True)
tx_cnt_date_temp = tx_cnt_date.merge(day_cross_bank_ratio, on=['cust_id','date'], how='left')
tx_cnt_date_temp.day_cross_bank_ratio = tx_cnt_date_temp.day_cross_bank_ratio / tx_cnt_date_temp.tx_cnt_date
base3 = base3.merge(tx_cnt_date_temp.drop('tx_cnt_date',axis = 1), on=['cust_id','date'], how='left')
base3[['day_cross_bank_ratio']] = base3[['day_cross_bank_ratio']].fillna(0)

In [16]:
train_data = base3[~base3.sar_flag.isna()].reset_index()
test_data = base3[base3.sar_flag.isna()].reset_index()

In [29]:
train_col = ['total_asset','occupation_code_tge','AGE_tge', 'lupay', 'cycam', 'usgam',
             'feature_01', 'feature_02', 'feature_03','feature_04', 'feature_05', 'risk_rank_0',
             'risk_rank_1', 'risk_rank_2','risk_rank_3','session_amt_diff_ratio', 'date_amt_diff_ratio',
             'tx_cnt_date', 'tx_cnt_session', 'txbranch_day_cnt','txbranch_session_cnt', 'session_atm_txn_ratio',
             'day_atm_txn_ratio','session_cross_bank_ratio', 'day_cross_bank_ratio']
test_size = 0.2
seed = 42
X_train, X_val, y_train, y_val = train_test_split(train_data[train_col], train_data['sar_flag'], test_size=test_size, random_state=seed)
imb_weight = y_train.value_counts()[0]/y_train.value_counts()[1]
model = XGBClassifier(eta = 0.01,max_depth = 4, scale_pos_weight = imb_weight)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_val, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(precision_score(y_val, predictions))
print(recall_score(y_val, predictions))

[20:44:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 81.36%
0.08333333333333333
0.6341463414634146


In [18]:
test_data['probability'] = model.predict_proba(test_data[train_col])[:,1]
test_data

,index,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,byymm,...,date_amt_diff_ratio,tx_cnt_date,tx_cnt_session,txbranch_day_cnt,txbranch_session_cnt,session_atm_txn_ratio,day_atm_txn_ratio,session_cross_bank_ratio,day_cross_bank_ratio,probability
0,8073,361270,389,NaN,0172056578071e83399216fcd640bdc4de4583149d0fd9...,1,19.0,2924860.0,5,365,...,1.000000,7.0,8.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.252349
1,8074,358040,382,NaN,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,1,19.0,493333.0,4,365,...,-999.000000,0.0,7.0,0.0,6.0,0.857143,0.0,0.000000,0.000000,0.343938
2,8075,360990,389,NaN,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,3,12.0,703297.0,5,365,...,1.000000,4.0,4.0,2.0,2.0,1.000000,1.0,0.000000,0.000000,0.176692
3,8076,364148,383,NaN,01c705b457297805c1c08137cb3adb164b8f393c265db2...,3,19.0,390912.0,4,365,...,0.588202,358.0,2512.0,295.0,2133.0,1.000000,1.0,0.696258,0.684358,0.179673
4,8077,364289,383,NaN,022bec0a12b56285bf12c49ba1512c0acdbf4176a30d96...,1,12.0,65301.0,3,365,...,-999.000000,0.0,3.0,0.0,1.0,0.333333,0.0,0.000000,0.000000,0.256318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,8803,364767,393,NaN,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,2,17.0,41066.0,3,365,...,0.386497,25.0,50.0,5.0,15.0,1.000000,1.0,0.300000,0.200000,0.744637
731,8804,355182,375,NaN,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,1,18.0,824.0,3,365,...,-999.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.198335
732,8805,360492,386,NaN,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,1,19.0,2406614.0,6,365,...,0.558794,3.0,6.0,0.0,0.0,0.166667,0.0,0.000000,0.000000,0.259728
733,8806,356426,377,NaN,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,3,19.0,21647.0,4,365,...,1.000000,3.0,19.0,3.0,18.0,0.947368,1.0,0.789474,0.000000,0.376525


In [22]:
base_f = base[base.sar_flag.isna()].merge(test_data[['cust_id','date','probability']],on = ['cust_id','date'], how = 'left')[['alert_key','probability']]
base_f.loc[base_f[base_f.probability.isna()].index,'probability'] = 0

In [23]:
doc_f = doc[~doc.alert_key.isin(base_f.alert_key.tolist())]
doc_f.loc[:,'probability'] = 0

C:\Users\Jasonhsu\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [25]:
pd.concat([base_f,doc_f]).to_csv('./submission_jason_v1.csv',index = False)

In [31]:
pd.DataFrame({'columns':list(X_train.columns),'score':model.feature_importances_}).sort_values('score',ascending = False)

,columns,score
18,tx_cnt_session,0.213341
14,risk_rank_3,0.149363
20,txbranch_session_cnt,0.129170
21,session_atm_txn_ratio,0.058128
12,risk_rank_1,0.044810
9,feature_04,0.038896
4,cycam,0.037706
22,day_atm_txn_ratio,0.037472
0,total_asset,0.036449
24,day_cross_bank_ratio,0.031505


In [57]:
base[base.alert_key == 361133]['cust_id'].value_counts()

067beb2294918c7d18f877a415e7ca9dc67723d1eac562094da1dc16270615f0    1
Name: cust_id, dtype: int64

In [58]:
base[base.cust_id == '067beb2294918c7d18f877a415e7ca9dc67723d1eac562094da1dc16270615f0']

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,byymm
9529,238306,151,0.0,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,122
10974,251069,179,0.0,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,153
16801,298824,270,0.0,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,244
18891,313154,291,0.0,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,275
21862,344674,334,0.0,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,334
23197,346536,354,0.0,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,334
23670,349629,361,0.0,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,334
24926,358178,382,NaN,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,365
25390,361133,389,NaN,067beb2294918c7d18f877a415e7ca9dc67723d1eac562...,1,14.0,0.0,4,365


In [59]:
dp[dp.cust_id == '067beb2294918c7d18f877a415e7ca9dc67723d1eac562094da1dc16270615f0']

,cust_id,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM
